In [1]:
from functools import reduce

from cycler import cycler
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
from numbers import Number
import numpy as np
import pandas as pd
#from scipy import stats as sstats
import wmfdata as wmf
from wmfdata.utils import pct_str, sig_figs
from wmfdata.charting import pct_fmt, M_fmt, comma_fmt

import gspread
from google.oauth2.service_account import Credentials

In [2]:
FILE = "metrics/metrics.tsv"
metrics = pd.read_csv(FILE, sep="\t", parse_dates=["month"]).set_index("month")

# Report

In [3]:
report_order = [
    'structured_data_used', 
    'num_commons_files_used_content_pages',
    'wikidata_items',
    'wikidata_items_being_reused'
]

In [4]:
new_metrics = metrics.iloc[[-1]].transpose().reindex(report_order).fillna("—")

In [5]:
new_metrics.at['structured_data_used','status'] =  new_metrics.iloc[0][0]/87000000-1
new_metrics.at['num_commons_files_used_content_pages','status'] = new_metrics.iloc[1][0]/31200000-1
new_metrics.at['wikidata_items_being_reused','status'] = new_metrics.iloc[3][0]/new_metrics.iloc[2][0]

In [6]:
#March
new_metrics

month,2022-06-01 00:00:00,status
structured_data_used,131146802.0,0.507435
num_commons_files_used_content_pages,38636121.0,0.238337
wikidata_items,97778126.0,NaN
wikidata_items_being_reused,20903956.0,0.213790


In [7]:
new_metrics

month,2022-06-01 00:00:00,status
structured_data_used,131146802.0,0.507435
num_commons_files_used_content_pages,38636121.0,0.238337
wikidata_items,97778126.0,NaN
wikidata_items_being_reused,20903956.0,0.213790


# Update Google Sheet

In [8]:
FILE = "metrics/metrics.tsv"
all_metrics = pd.read_csv(FILE, sep="\t")

In [9]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = Credentials.from_service_account_file('key-metrics-343717-07427b76df93.json', scopes=scope)
gc = gspread.authorize(credentials)

In [10]:
all_metrics.iloc[-1,1:] = all_metrics.iloc[-1,1:].astype(float)

In [11]:
@property
def sheet1(self):
    """Shortcut property for getting the first worksheet."""
    return self.get_worksheet(0)

In [12]:
wks1 = gc.open("platform evolution/structured data metrics").sheet1
new_metrics = all_metrics.iloc[-1,:].tolist()
wks1.append_row(new_metrics)

{'spreadsheetId': '12pI5UtJ9QBsFWxVjmRVJj6bBdwk_bwxFPmS-APLSQL4',
 'tableRange': 'Sheet1!A1:E25',
 'updates': {'spreadsheetId': '12pI5UtJ9QBsFWxVjmRVJj6bBdwk_bwxFPmS-APLSQL4',
  'updatedRange': 'Sheet1!A26:E26',
  'updatedRows': 1,
  'updatedColumns': 5,
  'updatedCells': 5}}